# Deployment with Gradio

In [1]:
!nvidia-smi

Fri May 14 06:25:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import files
files.upload() #upload kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
import time
time.sleep(5)
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d noulam/tomato
!unzip -q /content/tomato.zip

Saving kaggle.json to kaggle.json
kaggle.json
 98% 729M/743M [00:18<00:00, 29.0MB/s]
100% 743M/743M [00:18<00:00, 41.1MB/s]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
# using Densenet for Gradio deployment

from keras.models import load_model
model = load_model('/content/drive/MyDrive/Tomato_DenseNet121_10.h5')


In [ ]:
!pip install gradio

In [63]:

import gradio as gr
import tensorflow as tf
import numpy as np
import requests
import cv2
from keras.models import load_model
from keras.applications.densenet import preprocess_input

targets = ['Bacterial_spot',
          'Early_blight',
          'Late_blight',
          'Leaf_Mold',
          'Septoria_leaf_spot',
          'Spider_mites Two-spotted_spider_mite',
          'Target_Spot',
          'Tomato_Yellow_Leaf_Curl_Virus',
          'Tomato_mosaic_virus',
          'healthy']

def classify_image(img):
    
    img = cv2.resize(img, (256, 256))
    img = img/255.0     # if u have used 255.0 divison to normalize images while training
    # img = preprocess_input(img)   # if u have used preprocess_input to normalize images while training 
    img = np.expand_dims(img,axis=0)
    pred = densenet.predict(img)
    return  targets[pred.argmax()], round(pred[0][pred.argmax()]*100,2)
    
image = gr.inputs.Image() #shape = (256, 256, 3)) 

label = gr.outputs.Label(num_top_classes = 10)

gr.Interface(fn = classify_image, inputs = image, 
             outputs = label, capture_session = True).launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://47929.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://47929.gradio.app')